## Introduction 

The purpose of this model is to identify the most likely path for all users 

In [ ]:
import os
import pandas as pd 
import numpy as np
from pprint import pprint 

In [ ]:
def generate_dic(items): 
    """Genderates a dictionay for all values in item """
    new_dic = {}
    count = 0
    for unique in items: 
        new_dic[unique]  = count
        count +=1
        
    return new_dic

def transition_matrix(transitions):
    """
    Calculates a probability transtion matrix, and return a pandas dataframe with all the probabilities. 
    """
    unique_items = pd.unique(transitions)
    length = len(unique_items)
    transition_matrix = np.zeros((length, length))
    
    new_dict = generate_dic(unique_items)
    
    for (i,j) in zip(transitions,transitions[1:]):
        transition_matrix[new_dict[i],new_dict[j]] +=1
    
    transition_prob = pd.DataFrame(data = 0,index=unique_items, columns=unique_items)
    
    for column in range(0,len(unique_items)):
        for row in range(0,len(unique_items)):
            transition_prob.iloc[column,row]  = transition_matrix[column,row] /  np.sum(transition_matrix[column,0:])
        
    return round(transition_prob,3)

def _get_markov_edges(Q):
    """Takes a probabiilty matrix and outputs a dictionary like item for all the edges"""
    edges = {}
    for col in Q.columns:
        for idx in Q.index:
            edges[(idx,col)] = Q.loc[idx,col]
    return edges

In [ ]:
# set working directory (Change for youe)
os.chdir('C:/Users/ander/Google Drive/Columbia/Fall 2019/Capstone/Dotin-Columbia-Castone-Team-Alpha-')

In [ ]:
data = pd.read_csv('Models/Q1_Mouse Activity/Data/direction_data.csv')
data = data.drop(columns= 'Unnamed: 0')
data.head()

In [5]:
# calculate the proportion of each movement direction in the data
proportion = pd.DataFrame(data['Direction'].value_counts())
proportion['Proportion'] = proportion['Direction'] /proportion['Direction'].sum()
proportion

,Direction,Proportion
No Movement,1551783,0.191068
North,1043138,0.128439
East,959875,0.118187
North East,935212,0.115151
South East,845577,0.104114
West,823033,0.101338
South West,769229,0.094714
South,612332,0.075395
North West,581457,0.071594


In [6]:
# identify all the states
states_ = pd.unique(data['Direction'])
states_

array(['North East', 'North West', 'No Movement', 'North', 'South East',
       'South West', 'South', 'West', 'East'], dtype=object)

In [7]:
user_direction = data[['User Id', 'Direction']]

In [8]:
# create a list for all the directions in the dataset 
direction_all = []
for value in data['Direction']: 
    direction_all.append(value)

In [9]:
# calculate the probability matrix
prob_matrix = transition_matrix(direction_all)
prob_matrix

,North East,North West,No Movement,North,South East,South West,South,West,East
North East,0.585,0.005,0.161,0.100,0.004,0.011,0.004,0.005,0.124
North West,0.007,0.576,0.134,0.003,0.005,0.007,0.102,0.005,0.162
No Movement,0.099,0.047,0.385,0.112,0.078,0.071,0.062,0.067,0.079
North,0.099,0.004,0.146,0.569,0.099,0.002,0.006,0.039,0.035
South East,0.004,0.003,0.137,0.116,0.600,0.004,0.004,0.128,0.003
South West,0.014,0.005,0.142,0.009,0.004,0.630,0.086,0.107,0.003
South,0.005,0.097,0.179,0.007,0.002,0.102,0.511,0.048,0.048
West,0.002,0.003,0.136,0.045,0.121,0.108,0.039,0.536,0.011
East,0.112,0.102,0.132,0.036,0.002,0.003,0.033,0.010,0.570


In [10]:
# find all the edges for the probability matrix 
edges_wts =  _get_markov_edges(prob_matrix)
pprint(edges_wts)

{('East', 'East'): 0.57,
 ('East', 'No Movement'): 0.132,
 ('East', 'North'): 0.036,
 ('East', 'North East'): 0.112,
 ('East', 'North West'): 0.102,
 ('East', 'South'): 0.033,
 ('East', 'South East'): 0.002,
 ('East', 'South West'): 0.003,
 ('East', 'West'): 0.01,
 ('No Movement', 'East'): 0.079,
 ('No Movement', 'No Movement'): 0.385,
 ('No Movement', 'North'): 0.112,
 ('No Movement', 'North East'): 0.099,
 ('No Movement', 'North West'): 0.047,
 ('No Movement', 'South'): 0.062,
 ('No Movement', 'South East'): 0.078,
 ('No Movement', 'South West'): 0.071,
 ('No Movement', 'West'): 0.067,
 ('North', 'East'): 0.035,
 ('North', 'No Movement'): 0.146,
 ('North', 'North'): 0.569,
 ('North', 'North East'): 0.099,
 ('North', 'North West'): 0.004,
 ('North', 'South'): 0.006,
 ('North', 'South East'): 0.099,
 ('North', 'South West'): 0.002,
 ('North', 'West'): 0.039,
 ('North East', 'East'): 0.124,
 ('North East', 'No Movement'): 0.161,
 ('North East', 'North'): 0.1,
 ('North East', 'North East

In [13]:
# create state space and initial state probabilities
hidden_states = np.array(proportion.reset_index()['index'])

In [14]:
pi = np.array(proportion['Proportion'])
state_space = pd.Series(pi, index=hidden_states, name='states')
state_space

No Movement    0.191068
North          0.128439
East           0.118187
North East     0.115151
South East     0.104114
West           0.101338
South West     0.094714
South          0.075395
North West     0.071594
Name: states, dtype: float64

In this situation the true state of the user is unknown, thus hidden from you. One way to model this is to assume that the user has observable behaviors that represent the true, hidden state. Let's walk through an example.

First we create our state space - accurate or fraud. We assume they are equiprobable. 

In [15]:
# create hidden transition matrix
# a or alpha 
#   = transition probability matrix of changing states given a state
# matrix is size (M x M) where M is number of states
alpha_paramter =  np.array(prob_matrix)
alpha_paramter

array([[0.585, 0.005, 0.161, 0.1  , 0.004, 0.011, 0.004, 0.005, 0.124],
       [0.007, 0.576, 0.134, 0.003, 0.005, 0.007, 0.102, 0.005, 0.162],
       [0.099, 0.047, 0.385, 0.112, 0.078, 0.071, 0.062, 0.067, 0.079],
       [0.099, 0.004, 0.146, 0.569, 0.099, 0.002, 0.006, 0.039, 0.035],
       [0.004, 0.003, 0.137, 0.116, 0.6  , 0.004, 0.004, 0.128, 0.003],
       [0.014, 0.005, 0.142, 0.009, 0.004, 0.63 , 0.086, 0.107, 0.003],
       [0.005, 0.097, 0.179, 0.007, 0.002, 0.102, 0.511, 0.048, 0.048],
       [0.002, 0.003, 0.136, 0.045, 0.121, 0.108, 0.039, 0.536, 0.011],
       [0.112, 0.102, 0.132, 0.036, 0.002, 0.003, 0.033, 0.01 , 0.57 ]])

In [17]:
# create matrix of observation (emission) probabilities
# b or beta = observation probabilities given state
# matrix is size (M x O) where M is number of states 
# and O is number of different possible observations
b_df = pd.DataFrame(columns=states_, index=hidden_states)
for row in range(0, len(hidden_states)): 
    for column in range(0, len(hidden_states)):
        b_df.iloc[row, column] = 1/ len(hidden_states)

b = b_df.values.T
b

array([[0.1111111111111111, 0.1111111111111111, 0.1111111111111111,
        0.1111111111111111, 0.1111111111111111, 0.1111111111111111,
        0.1111111111111111, 0.1111111111111111, 0.1111111111111111],
       [0.1111111111111111, 0.1111111111111111, 0.1111111111111111,
        0.1111111111111111, 0.1111111111111111, 0.1111111111111111,
        0.1111111111111111, 0.1111111111111111, 0.1111111111111111],
       [0.1111111111111111, 0.1111111111111111, 0.1111111111111111,
        0.1111111111111111, 0.1111111111111111, 0.1111111111111111,
        0.1111111111111111, 0.1111111111111111, 0.1111111111111111],
       [0.1111111111111111, 0.1111111111111111, 0.1111111111111111,
        0.1111111111111111, 0.1111111111111111, 0.1111111111111111,
        0.1111111111111111, 0.1111111111111111, 0.1111111111111111],
       [0.1111111111111111, 0.1111111111111111, 0.1111111111111111,
        0.1111111111111111, 0.1111111111111111, 0.1111111111111111,
        0.1111111111111111, 0.11111111111111

In [18]:
# define Viterbi algorithm for shortest path
# code adapted from Stephen Marsland's, Machine Learning An Algorthmic Perspective, Vol. 2
# https://github.com/alexsosn/MarslandMLAlgo/blob/master/Ch16/HMM.py

def viterbi(pi, a, b, obs):
    
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]
    
    # init blank path
    path = np.zeros(T,dtype=int)
    # delta --> highest probability of any path that reaches state i
    delta = np.zeros((nStates, T))
    # phi --> argmax by time step for each state
    phi = np.zeros((nStates, T))
    
    # init delta and phi 
    delta[:, 0] = pi * b[:, int(obs[0])]
    phi[:, 0] = 0

   # print('\nStart Walk Forward\n')    
    # the forward algorithm extension
    for t in range(1, T):
        for s in range(nStates):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, int(obs[t])] 
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])
            #print('s={s} and t={t}: phi[{s}, {t}] = {phi}'.format(s=s, t=t, phi=phi[s, t]))
    
    # find optimal path
    print('-'*50)
    #print('Start Backtrace\n')
    path[T-1] = np.argmax(delta[:, T-1])
    #p('init path\n    t={} path[{}-1]={}\n'.format(T-1, T, path[T-1]))
    for t in range(T-2, -1, -1):
        path[t] = phi[path[t+1], [t+1]]
        #p(' '*4 + 't={t}, path[{t}+1]={path}, [{t}+1]={i}'.format(t=t, path=path[t+1], i=[t+1]))
        #print('path[{}] = {}'.format(t, path[t]))
        
    return path, delta, phi

In [19]:

direction_dict =  generate_dic(pd.unique(user_365_direction))
direction_dict

{'North East': 0,
 'North West': 1,
 'No Movement': 2,
 'North': 3,
 'South East': 4,
 'South West': 5,
 'South': 6,
 'West': 7,
 'East': 8}

In [45]:
unique_id = pd.unique(data['User Id'])

In [27]:
user_365_direction =data[data['User Id'] == 365]['Direction']

direction_365 = []
for value in user_365_direction: 
    direction_365 = np.append([direction_365],[direction_dict[value]])

In [69]:
prob_value_matrix = []
user_matrix = []
for user in unique_id: 
    user_direction = data[data['User Id'] == user]['Direction']
    user_direction_all = []
    for value in user_direction: 
            user_direction_all = np.append([user_direction_all],[direction_dict[value]])     
    
    path, delta, phi = viterbi(pi, alpha_paramter, b, user_direction_all[1:200])
            
    prob_value = np.sum(delta)
            
    prob_value_matrix.append(prob_value)
    user_matrix.append(user)


Start Walk Forward

--------------------------------------------------
Start Backtrace


Start Walk Forward

--------------------------------------------------
Start Backtrace


Start Walk Forward

--------------------------------------------------
Start Backtrace


Start Walk Forward

--------------------------------------------------
Start Backtrace


Start Walk Forward

--------------------------------------------------
Start Backtrace


Start Walk Forward

--------------------------------------------------
Start Backtrace


Start Walk Forward

--------------------------------------------------
Start Backtrace


Start Walk Forward

--------------------------------------------------
Start Backtrace


Start Walk Forward

--------------------------------------------------
Start Backtrace


Start Walk Forward

--------------------------------------------------
Start Backtrace


Start Walk Forward

--------------------------------------------------
Start Backtrace


Start Walk Forward



KeyboardInterrupt: 

In [70]:
prob_value_matrix

[0.11839009512851356,
 0.11839009512851356,
 0.11839009512851356,
 0.11839009512851356,
 0.11839009512851356,
 0.11839009512851356,
 0.11839009512851356,
 0.11839009512851356,
 0.11839009512851356,
 0.11839009512851356,
 0.11839009512851356,
 0.11839009512851356,
 0.11839009512851356]

In [75]:
prob_value_user = []
prob_matrix = []

path

array([5, 5, 5, ..., 0, 0, 0])

In [57]:
prob_ = np.sum(delta)

prob_value = pd.DataFrame({'User ID': user, 
                          'Prob': prob_})

ValueError: If using all scalar values, you must pass an index

In [71]:
path, delta, phi = viterbi(pi, alpha_paramter, b, direction_365)
print('\nsingle best state path: \n', path)
print('delta:\n', delta)
print('phi:\n', phi)


Start Walk Forward

--------------------------------------------------
Start Backtrace


single best state path: 
 [5 5 5 ... 0 0 0]
delta:
 [[2.12297539e-02 1.37993400e-03 8.96957101e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.42710437e-02 9.13346796e-04 5.84541950e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.31319328e-02 5.61754901e-04 2.46854860e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.05237281e-02 5.97513895e-04 3.39255111e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.37723938e-03 4.98911145e-04 2.97129304e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.95484227e-03 5.03806677e-04 3.19077562e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
phi:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 2. 0. ... 0. 0. 0.]
 ...
 [0. 6. 6. ... 0. 0. 0.]
 [0. 7. 7. ... 0. 0. 0.]
 [0. 8. 8. ... 0. 0. 0.]]


Sources: 

https://www.blackarbs.com/blog/introduction-hidden-markov-models-python-networkx-sklearn/2/9/2017 <b/>

https://github.com/alexsosn/MarslandMLAlgo/blob/master/Ch16/HMM.py